# Item-based Collaborative Filtering

## Import required libraries

In [19]:
# For reading data
import os
from urllib.parse import quote_plus
from dotenv import load_dotenv

# For processing data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# For connecting to the DB
from sqlalchemy import create_engine, text

# For data visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### ============================================================================
## Getting info from the database server
#### ============================================================================

### Gather necessary info

In [20]:
load_dotenv()
password = os.getenv('DB_PASSWORD') or input("Enter password: ")
host = os.getenv('DB_HOST', 'car-analytics-db.copkksw0o3bx.us-east-1.rds.amazonaws.com')
user = os.getenv('DB_USER', 'ds5230_postgres')

# Encode password and connect to the database
password_encoded = quote_plus(password)
engine = create_engine(f'postgresql://{user}:{password_encoded}@{host}:5432/car_analytics_db')

#### Check the connection and show the table

In [21]:
# Set up the query and print result

tables_query = """ select * from cars_ohe; """
cars_table = pd.read_sql(tables_query, engine)
cars_table

,price,year,odometer,lat,long,state_al,state_ar,state_az,state_ca,state_co,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
0,36990,2017,38094.0,284.000000,-81.960000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,27995,2006,NaN,42.477800,-92.366100,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,78423,2015,30200.0,38.401800,-93.785000,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,14000,2013,92965.0,40.320300,-80.625000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2744,2002,229031.0,42.457439,-92.332855,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8485,6999,2004,124737.0,25.490600,-80.438000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8486,15590,2016,53066.0,39.300000,-76.610000,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
8487,5500,1999,298000.0,40.759200,-124.159300,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
8488,4495,2008,132309.0,43.424800,-83.974500,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### ============================================================================
## Data Pre-processing
#### ============================================================================

### Checking and filling missing values

In [22]:
# Check for missing values in key numerical features
print("\nMissing values before handling:")
print(f"  odometer: {cars_table['odometer'].isna().sum()} ({cars_table['odometer'].isna().sum()/len(cars_table)*100:.1f}%)")
print(f"  lat: {cars_table['lat'].isna().sum()} ({cars_table['lat'].isna().sum()/len(cars_table)*100:.1f}%)")
print(f"  long: {cars_table['long'].isna().sum()} ({cars_table['long'].isna().sum()/len(cars_table)*100:.1f}%)")

# # Handle missing values in odometer column  by fill odometer with means
# cars_table.fillna({'odometer': cars_table['odometer'].mean()}, inplace=True)

# # Fill lat/long with mean values
# # Alternative: You could also fill with values based on the state
# cars_table.fillna({'lat': cars_table['lat'].mean()}, inplace=True)
# cars_table.fillna({'long': cars_table['long'].mean()}, inplace=True)

# numerical features
numeric_features = ['price', 'year', 'odometer', 'lat', 'long']

# Number of similar samples to use for prediction
n_neighbors = 5

# Convert boolean strings to integers
for col in cars_table.columns:
    if cars_table[col].dtype == 'object':
        cars_table[col] = cars_table[col].map({'True': 1, 'False': 0})

# Filling in missing value using cosine_similarity and neighbor clustering
for feature in numeric_features:
    # Identify rows with and without missing values
    missing_mask = cars_table[feature].isnull()
    complete_rows = cars_table[~missing_mask].copy()
    incomplete_rows = cars_table[missing_mask].copy()
    
    if len(incomplete_rows) != 0: 
        # Get all columns except the target
        feature_cols = [col for col in cars_table.columns if col != feature]
        
        # Prepare feature matrices (fill any other missing values with 0 for similarity calculation)
        X_complete = complete_rows[feature_cols].fillna(0).values
        X_incomplete = incomplete_rows[feature_cols].fillna(0).values
        y_complete = complete_rows[feature].values
        
        # Calculate cosine similarity between incomplete and complete rows
        similarities = cosine_similarity(X_incomplete, X_complete)
        
        # Predict missing values
        predictions = []
        for i, sim_scores in enumerate(similarities):
            # Get indices of top n_neighbors most similar samples
            top_n_indices = np.argsort(sim_scores)[-n_neighbors:]
            
            # Get the values from most similar samples
            neighbor_values = y_complete[top_n_indices]
            
            # For numerical values: weighted average based on similarity
            weights = sim_scores[top_n_indices]
            
            # Avoid division by zero
            if weights.sum() == 0:
                predicted_value = np.mean(neighbor_values)
            else:
                predicted_value = np.average(neighbor_values, weights=weights)
            
            predictions.append(predicted_value)
        
        # Fill the column up with predictions
        cars_table.loc[missing_mask, feature] = predictions



# Create a vehicle ID for tracking
cars_table['vehicle_id'] = range(len(cars_table))
cars_table


Missing values before handling:
  odometer: 54 (0.6%)
  lat: 79 (0.9%)
  long: 79 (0.9%)


C:\Users\patri\AppData\Local\Temp\ipykernel_17004\3805449042.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cars_table['vehicle_id'] = range(len(cars_table))


,price,year,odometer,lat,long,state_al,state_ar,state_az,state_ca,state_co,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow,vehicle_id
0,36990,2017,38094.000000,284.000000,-81.960000,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,27995,2006,67708.999976,42.477800,-92.366100,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,78423,2015,30200.000000,38.401800,-93.785000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,14000,2013,92965.000000,40.320300,-80.625000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,2744,2002,229031.000000,42.457439,-92.332855,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8485,6999,2004,124737.000000,25.490600,-80.438000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8485
8486,15590,2016,53066.000000,39.300000,-76.610000,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,8486
8487,5500,1999,298000.000000,40.759200,-124.159300,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,8487
8488,4495,2008,132309.000000,43.424800,-83.974500,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8488


### Feature Count

In [23]:
# Select numerical and categorical features for similarity computation
# We'll use a mix of continuous and binary features

# Binary features (one-hot encoded in the dataset)
# Get all state columns
state_cols = [col for col in cars_table.columns if col.startswith('state_')]
# Get all condition columns
condition_cols = [col for col in cars_table.columns if col.startswith('condition_')]
# Get all cylinder columns
cylinder_cols = [col for col in cars_table.columns if col.startswith('cylinders_')]
# Get all fuel columns
fuel_cols = [col for col in cars_table.columns if col.startswith('fuel_')]
# Get all manufacturer columns
manufacturer_cols = [col for col in cars_table.columns if col.startswith('manufacturer_')]
# Get all drive columns
drive_cols = [col for col in cars_table.columns if col.startswith('drive_')]
# Get all title_status columns
title_cols = [col for col in cars_table.columns if col.startswith('title_status_')]
# Get all paint_color columns
paint_cols = [col for col in cars_table.columns if col.startswith('paint_color_')]

# Combine all features
all_features = (numeric_features + state_cols + condition_cols + 
                cylinder_cols + fuel_cols + manufacturer_cols + 
                drive_cols + title_cols + paint_cols)

print(f"Total features for similarity: {len(all_features)}")
print(f"  - Numerical features: {len(numeric_features)}")
print(f"  - State features: {len(state_cols)}")
print(f"  - Condition features: {len(condition_cols)}")
print(f"  - Manufacturer features: {len(manufacturer_cols)}")
print(f"  - Other categorical features: {len(all_features) - len(numeric_features) - len(state_cols) - len(condition_cols) - len(manufacturer_cols)}")

Total features for similarity: 126
  - Numerical features: 5
  - State features: 50
  - Condition features: 5
  - Manufacturer features: 37
  - Other categorical features: 29


#### ============================================================================
## Helper Functions
#### ============================================================================

### Create user profile

In [24]:
def create_user_profile(preferences, columns_label):
    """
    Creates a user profile vector based on preferences.
    This vector will be used for similarity comparison.
    """
    # Initialize user profile with zeros
    user_profile = pd.Series(0, index=columns_label)
    
    # Handle numeric features (price, year, odometer)
    # We'll normalize them to match the scale of binary features
    scaler = StandardScaler()
    
    if 'price' in preferences:
        price_min, price_max = preferences['price']
        user_profile['price'] = (price_min + price_max) / 2
    
    if 'year' in preferences:
        year_min, year_max = preferences['year']
        user_profile['year'] = (year_min + year_max) / 2
    
    if 'odometer' in preferences:
        odo_min, odo_max = preferences['odometer']
        user_profile['odometer'] = (odo_min + odo_max) / 2
    
    # Handle categorical features (one-hot encoded)
    if 'state' in preferences:
        col_name = f"state_{preferences['state']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'condition' in preferences:
        col_name = f"condition_{preferences['condition']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'cylinders' in preferences:
        col_name = f"cylinders_{preferences['cylinders']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'fuel' in preferences:
        col_name = f"fuel_{preferences['fuel']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'manufacturer' in preferences:
        col_name = f"manufacturer_{preferences['manufacturer']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'drive' in preferences:
        col_name = f"drive_{preferences['drive']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'title_status' in preferences:
        col_name = f"title_status_{preferences['title_status']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    if 'paint_color' in preferences:
        col_name = f"paint_color_{preferences['paint_color']}"
        if col_name in user_profile.index:
            user_profile[col_name] = 1
    
    return user_profile

### Filter the dataset within user's specified ranges

In [25]:
def filter_by_range(df, preferences):
    """
    Filter vehicles that fall within user's specified ranges
    This is a hard filter before similarity calculation
    """
    filtered_df = df.copy()
    
    # Filter by price range
    if 'price' in preferences:
        price_min, price_max = preferences['price']
        filtered_df = filtered_df[
            (filtered_df['price'] >= price_min) & 
            (filtered_df['price'] <= price_max)
        ]
    
    # Filter by year range
    if 'year' in preferences:
        year_min, year_max = preferences['year']
        filtered_df = filtered_df[
            (filtered_df['year'] >= year_min) & 
            (filtered_df['year'] <= year_max)
        ]
    
    # Filter by odometer range
    if 'odometer' in preferences:
        odo_min, odo_max = preferences['odometer']
        filtered_df = filtered_df[
            (filtered_df['odometer'].notna()) &
            (filtered_df['odometer'] >= odo_min) & 
            (filtered_df['odometer'] <= odo_max)
        ]
    
    return filtered_df

### Convert OHE features back to readable format

In [26]:
def decode_vehicle_info(vehicle_row):
    """
    Decodes one-hot encoded features back to readable format.
    """
    info = {}
    
    # Basic info
    info['Price'] = f"${vehicle_row['price']:,.2f}"
    info['Year'] = int(vehicle_row['year'])
    info['Odometer'] = f"{vehicle_row['odometer']:,.0f} miles" if pd.notna(vehicle_row['odometer']) else "N/A"
    
    # Decode categorical features
    for col in vehicle_row.index:
        if vehicle_row[col] == True or vehicle_row[col] == 'True':
            if col.startswith('state_'):
                info['State'] = col.replace('state_', '').upper()
            elif col.startswith('condition_'):
                info['Condition'] = col.replace('condition_', '').title()
            elif col.startswith('cylinders_'):
                info['Cylinders'] = col.replace('cylinders_', '').title()
            elif col.startswith('fuel_'):
                info['Fuel'] = col.replace('fuel_', '').title()
            elif col.startswith('manufacturer_'):
                info['Manufacturer'] = col.replace('manufacturer_', '').title()
            elif col.startswith('drive_'):
                info['Drive'] = col.replace('drive_', '').upper()
            elif col.startswith('title_status_'):
                info['Title Status'] = col.replace('title_status_', '').title()
            elif col.startswith('paint_color_'):
                info['Color'] = col.replace('paint_color_', '').title()
    
    return info

#### ============================================================================
## Calculate simularities of the user input item with other items
#### ============================================================================

### Formatting User input

In [27]:
# Sample user input for demonstration
sample_preferences = {
    'price': (5000, 15000),
    'year': (2005, 2015),
    'fuel': 'gas',
    'manufacturer': 'ford',
    'paint_color': 'yellow'
}
# create a user profile
user_profile = create_user_profile(sample_preferences, cars_table.columns[:-1])
user_profile

price                 10000
year                   2010
odometer                  0
lat                       0
long                      0
                      ...  
paint_color_purple        0
paint_color_red           0
paint_color_silver        0
paint_color_white         0
paint_color_yellow        1
Length: 126, dtype: int64

### Filter out the vehicle based on the range preferences (price, year, odometer)

In [28]:
filtered_car_table = filter_by_range(cars_table, sample_preferences)
filtered_car_table

,price,year,odometer,lat,long,state_al,state_ar,state_az,state_ca,state_co,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow,vehicle_id
3,14000,2013,92965.0,4.032030e+01,-8.062500e+01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
9,10950,2010,150900.0,4.440501e+01,-8.881125e+01,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,9
10,11995,2007,192493.0,3.409929e+06,-1.174312e+02,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,10
15,13995,2013,156190.0,4.180180e+01,-8.148149e+06,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,15
18,7999,2009,89000.0,3.982190e+01,-7.576870e+01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8464,6900,2005,228000.0,3.997120e+01,-8.612450e+01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8464
8475,7495,2011,172301.0,3.774212e+01,-8.916376e+01,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,8475
8476,13923,2014,80262.0,4.476513e+06,-1.231502e+02,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8476
8480,9995,2007,154722.0,6.116167e+01,-1.498631e+02,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8480


### Normalizing the filtered car table

In [29]:
# Get necessary data to normalize into a new table
normalized_table = filtered_car_table[all_features].copy()

# Normalize values in the table
scaler = StandardScaler()
normalized_table[numeric_features] = scaler.fit_transform(normalized_table[numeric_features])

# Attach the vehicle id back to the normalize table for vehicle identification
# normalized_table['vehicle_id'] = filtered_car_table.iloc[:, -1].values

print(f"Normalized table shape: {normalized_table.shape}")
normalized_table

Normalized table shape: (2423, 126)


,price,year,odometer,lat,long,state_al,state_ar,state_az,state_ca,state_co,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
3,1.537542,0.833444,-0.467921,-0.251832,0.270749,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0.480009,-0.212353,0.413800,-0.251827,0.270746,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10,0.842344,-1.258151,1.046810,3.465528,0.270734,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15,1.535809,0.833444,0.494309,-0.251830,-3.149918,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18,-0.543198,-0.560953,-0.528265,-0.251832,0.270751,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8464,-0.924257,-1.955349,1.587195,-0.251832,0.270747,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8475,-0.717951,0.136246,0.739505,-0.251834,0.270746,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8476,1.510844,1.182043,-0.661250,4.628287,0.270732,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8480,0.148880,-1.258151,0.471967,-0.251809,0.270720,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Normalize the user input

In [30]:
# Normalize numeric features
numeric_cols = ['price', 'year', 'odometer', 'lat', 'long']


user_normalized = user_profile.copy().to_frame().T
# Reorder columns to match the specified order
user_normalized = user_normalized[normalized_table.columns[:-1]]

scaler = StandardScaler()
scaler.fit(cars_table[numeric_features])

user_normalized[numeric_features] = scaler.transform(user_normalized[numeric_features])
# normalized_table.values
user_normalized

,price,year,odometer,lat,long,state_al,state_ar,state_az,state_ca,state_co,...,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white
0,-0.655833,-0.170227,-0.448007,-0.212145,0.20551,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculate simularities

In [31]:
# Calculate the cosine similarity between the user and the rest of the database
user_vector = user_normalized.values.reshape(1, -1)
vehicles_vector = normalized_table.iloc[:, :-1].values
cosine_similarities = cosine_similarity(user_vector, vehicles_vector)[0]
cosine_similarities

array([ 0.04012994,  0.42003696, -0.20329184, ..., -0.20202275,
        0.30730416,  0.01912385], shape=(2423,))

#### ============================================================================
## Get recommendations
#### ============================================================================

### Get N recommendations based on user's preference

In [32]:
# N recommendations based on user's specification
user_rec_list_len = 10

# Get top N recommendations
top_indices = cosine_similarities.argsort()[-user_rec_list_len:][::-1]
top_similarities = cosine_similarities[top_indices]

# Get original indices in the main dataframe
original_indices = filtered_car_table.index[top_indices]
recommendations_list = cars_table.loc[original_indices].copy()
recommendations_list['similarity_score'] = top_similarities
recommendations_list

,price,year,odometer,lat,long,state_al,state_ar,state_az,state_ca,state_co,...,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow,vehicle_id,similarity_score
1962,7990,2008,109547.0,42.987906,-88.241072,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1962,0.790229
537,5500,2012,114000.0,31.538440,-100.425682,0,0,0,0,0,...,0,0,0,0,0,0,0,0,537,0.784792
229,5900,2010,145773.0,42.303000,-85.130400,0,0,0,0,0,...,0,0,0,0,0,0,0,0,229,0.770122
5817,5950,2010,107000.0,45125.000000,-84.672300,0,0,0,0,0,...,0,0,0,0,0,0,1,0,5817,0.769622
6218,5495,2009,144458.0,43.182400,-84.112200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6218,0.765908
2333,5995,2010,149971.0,45.179871,-89.664241,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2333,0.759208
1437,5900,2011,106466.0,33.873436,-117.542806,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1437,0.758267
7398,6995,2013,91758.0,27.999033,-82.459702,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7398,0.757820
5009,5999,2011,110000.0,42.244100,-71.990600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5009,0.753857
2156,7995,2009,146814.0,44.307057,-93.289528,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2156,0.737208


### Display the recommendation list in a more readable format

In [33]:
if recommendations_list is None or len(recommendations_list) == 0:
    print("There are no vehicle that match your preference in our database!")
else:
    print("\n" + "=" * 80)
    print("TOP VEHICLE RECOMMENDATIONS")
    print("=" * 80)

    for idx, (_, vehicle) in enumerate(recommendations_list.iterrows(), 1):
        info = decode_vehicle_info(vehicle)
        similarity = vehicle['similarity_score'] * 100
        
        print(f"\n{'─' * 80}")
        print(f"RECOMMENDATION #{idx} - Match Score: {similarity:.1f}%")
        print(f"{'─' * 80}")
        
        for key, value in info.items():
            print(f"{key:.<20} {value}")
        
    print("\n" + "=" * 80)


TOP VEHICLE RECOMMENDATIONS

────────────────────────────────────────────────────────────────────────────────
RECOMMENDATION #1 - Match Score: 79.0%
────────────────────────────────────────────────────────────────────────────────
Price............... $7,990.00
Year................ 2008
Odometer............ 109,547 miles
State............... WI
Fuel................ Gas
Manufacturer........ Ford

────────────────────────────────────────────────────────────────────────────────
RECOMMENDATION #2 - Match Score: 78.5%
────────────────────────────────────────────────────────────────────────────────
Price............... $5,500.00
Year................ 2012
Odometer............ 114,000 miles
State............... TX
Fuel................ Gas
Manufacturer........ Ford

────────────────────────────────────────────────────────────────────────────────
RECOMMENDATION #3 - Match Score: 77.0%
────────────────────────────────────────────────────────────────────────────────
Price............... $5,900.00
